In [2]:
# TODO download a pretrained model
# TODO evaluate the model with a sample image
# TODO create ONNX model out of it with dynamic axis
# TODO apply ONNX level optimizations
# TODO evaluate the sanity of ONNX model
# TODO fine-tune with inference optimizer 

# Note: One should select the conda environment located at /opt/conda

In [16]:
import torch
import torchvision.models as models
model = models.resnet50(pretrained=True)


In [17]:
model.eval()
dummy_input = torch.randn(1,3,224, 224)
input_names = ["input_names"]
output_names = ["output"]

In [18]:
torch.onnx.export(model, dummy_input, "static-resnet50.onnx", verbose=False, input_names=input_names, output_names=output_names, export_params=True)

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [20]:
# dyanmic batch size
batch_size = 3 #random value
use_cuda =torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model.to(device)
dummy_input = torch.randn(batch_size, 3, 224, 224).to(device)
input_names = ["dynamic_input"]
output_names = ["output"]
dynamic_axes = {'dynamic_input':{0:'batch_size'},
               'output':{0:'batch_size'} } 

In [24]:
torch.onnx.export(model, dummy_input, "dynamic-resnet50.onnx", do_constant_folding=True, input_names=input_names, 
output_names=output_names, dynamic_axes=dynamic_axes
)

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [23]:
import onnx

In [26]:
model = onnx.load('dynamic-resnet50.onnx')
model.graph.input[0].type.tensor_type.shape.dim[0].dim_param = '?'
onnx.save(model, 'dynamic_restnet50-tuned.onnx')
onnx.checker.check_model(model)
